In [12]:
import requests
import urllib

CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'
KEY = 'dc70509bd505df1e1d0ebb190d894984'

url = CONFIG_PATTERN.format(key=KEY)
r = requests.get(url)
config = r.json()

In [13]:
def imdb_id_from_title(title):
    """ return IMDB id for search string
        Args::
            title (str): the movie title search string
        Returns: 
            str. IMDB id
            None. If no match was found
    """
    pattern = 'http://www.imdb.com/xml/find?json=1&nr=1&tt=on&q={movie_title}'
    url = pattern.format(movie_title=urllib.quote(title))
    r = requests.get(url)
    res = r.json()
    # sections in descending order or preference
    for section in ['popular','exact','substring']:
        key = 'title_' + section 
        if key in res:
            return res[key][0]['id']

In [7]:
imdbid = imdb_id_from_title("Titanic")

In [8]:
base_url = config['images']['base_url']
sizes = config['images']['poster_sizes']
def size_str_to_int(x):
    return float("inf") if x == 'original' else int(x[1:])
max_size = max(sizes, key=size_str_to_int)

In [9]:
IMG_PATTERN = 'http://api.themoviedb.org/3/movie/{imdbid}/images?api_key={key}' 
r = requests.get(IMG_PATTERN.format(key=KEY,imdbid=imdbid))
api_response = r.json()

In [10]:
posters = api_response['posters']
poster_urls = []
for poster in posters:
    rel_path = poster['file_path']
    url = "{0}{1}{2}".format(base_url, max_size, rel_path)
    poster_urls.append(url)

In [11]:
for nr, url in enumerate(poster_urls):
    r = requests.get(url)
    filetype = r.headers['content-type'].split('/')[-1]
    filename = 'poster_{0}.{1}'.format(nr+1,filetype) 
    with open(filename,'wb') as w:
        w.write(r.content)